# Netflix Content Analysis: Insights into TV Shows and Movies

In [5]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interactive, HBox, VBox
import numpy as np
import plotly.graph_objects as go
import plotly.colors as pc

# Load the dataset
df = pd.read_csv(r"C:\Users\pzx12\OneDrive\Desktop\MDMA Y1S2\MECG11303\DV A2\titles.csv")

# Check for missing values
missing_values = df.isnull().sum()

# Calculate percentage of missing values
missing_percentage = (missing_values / len(df)) * 100

# Create a DataFrame to display both count and percentage
missing_data = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percentage
})

###missing_data

# Delete the 'age_certification' column
df = df.drop(columns=['age_certification'])

# Drop rows with missing values in specific columns
df = df.dropna(subset=['imdb_score', 'imdb_votes', 'tmdb_score'])

# Convert the 'release_year' column to a datetime format with the year only
df['release_year'] = pd.to_datetime(df['release_year'], format='%Y').dt.year

# Normalize 'genres' and 'production_countries'
df['genres'] = df['genres'].apply(eval)
df['production_countries'] = df['production_countries'].apply(eval)

# Remove rows where 'genres' or 'production_countries' contain empty lists
df = df[~(df['genres'].apply(lambda x: x == []) | df['production_countries'].apply(lambda x: x == []))]




# Standardize country names
country_name_mapping = {
    'US': 'United States', 'GB': 'United Kingdom', 'EG': 'Egypt', 'DE': 'Germany', 
    'IN': 'India', 'SU': 'Soviet Union', 'LB': 'Lebanon', 'CA': 'Canada', 'FR': 'France', 
    'DZ': 'Algeria', 'IT': 'Italy', 'JP': 'Japan', 'AR': 'Argentina', 'PE': 'Peru', 
    'BR': 'Brazil', 'IE': 'Ireland', 'HK': 'Hong Kong', 'AU': 'Australia', 'GH': 'Ghana', 
    'BF': 'Burkina Faso', 'MX': 'Mexico', 'ES': 'Spain', 'PS': 'Palestine', 'NO': 'Norway', 
    'BE': 'Belgium', 'DK': 'Denmark', 'BS': 'Bahamas', 'CZ': 'Czech Republic', 'MT': 'Malta', 
    'ZA': 'South Africa', 'CN': 'China', 'RU': 'Russia', 'KR': 'South Korea', 'CO': 'Colombia', 
    'CL': 'Chile', 'MA': 'Morocco', 'UY': 'Uruguay', 'TR': 'Turkey', 'NL': 'Netherlands', 
    'SE': 'Sweden', 'TW': 'Taiwan', 'NG': 'Nigeria', 'MY': 'Malaysia', 'CH': 'Switzerland', 
    'PH': 'Philippines', 'KW': 'Kuwait', 'AT': 'Austria', 'AE': 'United Arab Emirates', 
    'HU': 'Hungary', 'ID': 'Indonesia', 'SA': 'Saudi Arabia', 'SG': 'Singapore', 
    'FI': 'Finland', 'IL': 'Israel', 'CD': 'Congo', 'PL': 'Poland', 'VE': 'Venezuela', 
    'RO': 'Romania', 'IS': 'Iceland', 'UA': 'Ukraine', 'NZ': 'New Zealand', 'LU': 'Luxembourg', 
    'BG': 'Bulgaria', 'IR': 'Iran', 'JO': 'Jordan', 'QA': 'Qatar', 'HR': 'Croatia', 
    'TN': 'Tunisia', 'GL': 'Greenland', 'PT': 'Portugal', 'PK': 'Pakistan', 'VN': 'Vietnam', 
    'PR': 'Puerto Rico', 'IQ': 'Iraq', 'TH': 'Thailand', 'GE': 'Georgia', 'KH': 'Cambodia', 
    'CU': 'Cuba', 'VA': 'Vatican City', 'SY': 'Syria', 'RS': 'Serbia', 'TZ': 'Tanzania', 
    'AL': 'Albania', 'PY': 'Paraguay', 'ZW': 'Zimbabwe', 'NP': 'Nepal', 'CM': 'Cameroon', 
    'BD': 'Bangladesh', 'BT': 'Bhutan', 'LT': 'Lithuania', 'MC': 'Monaco', 'MW': 'Malawi', 
    'CY': 'Cyprus', 'SN': 'Senegal', 'LK': 'Sri Lanka', 'MU': 'Mauritius', 'IO': 'British Indian Ocean Territory', 
    'GT': 'Guatemala', 'AO': 'Angola', 'GR': 'Greece', 'KN': 'Saint Kitts and Nevis', 
    'KE': 'Kenya', 'FO': 'Faroe Islands', 'XX': 'Unknown'
}

# Apply the mapping to the 'production_countries' column
df['production_countries'] = df['production_countries'].apply(lambda countries: [country_name_mapping.get(country, country) for country in countries])

# Extract unique countries
unique_countries = np.unique([country for sublist in df['production_countries'] for country in sublist])


# Function to create a bar chart for genres count
def bar_genre(types, start_year, end_year, countries):
    df_filtered = df[
        (df['type'].isin(types)) &
        (df['release_year'] >= start_year) &
        (df['release_year'] <= end_year) &
        (df['production_countries'].apply(lambda x: any(item in x for item in countries)))
    ]
    genres_count = df_filtered['genres'].explode().value_counts()

    # Create a DataFrame for Plotly Express
    genres_df = genres_count.reset_index()
    genres_df.columns = ['Genre', 'Count']

    # Create the bar chart with colors
    fig = px.bar(genres_df, x='Genre', y='Count', color='Genre',
                 labels={'Genre': 'Genre', 'Count': 'Count'},
                 title=f'Genres Distribution from {start_year} to {end_year}')

    # Update layout with custom figure size
    fig.update_layout(
        xaxis_title='',
        width=1450,  # Set the width of the figure
        height=550  # Set the height of the figure
    )

    # Remove the x-axis tick labels
    fig.update_xaxes(showticklabels=False)

    fig.show()


# Function to create a line chart for the average runtime each year
def line_avg_runtime(types, start_year, end_year, genres, countries):
    df_filtered = df[
        (df['type'].isin(types)) &
        (df['release_year'] >= start_year) &
        (df['release_year'] <= end_year) &
        (df['genres'].apply(lambda x: any(item in x for item in genres))) &
        (df['production_countries'].apply(lambda x: any(item in x for item in countries)))
    ]
    avg_runtime_per_year = df_filtered.groupby('release_year')['runtime'].mean()

    # Create the line chart
    fig = go.Figure()

    # Add the line trace
    fig.add_trace(go.Scatter(x=avg_runtime_per_year.index, y=avg_runtime_per_year.values,
                             mode='lines+markers',
                             line=dict(color='royalblue', width=2),
                             marker=dict(size=8, color='red', line=dict(width=2, color='DarkSlateGrey')),
                             name='Average Runtime'))

    # Add a trendline (optional, using a simple linear fit)
    trendline = np.polyfit(avg_runtime_per_year.index, avg_runtime_per_year.values, 1)
    trendline_eq = np.poly1d(trendline)
    fig.add_trace(go.Scatter(x=avg_runtime_per_year.index, y=trendline_eq(avg_runtime_per_year.index),
                             mode='lines',
                             line=dict(dash='dash', color='green'),
                             name='Trendline'))

    # Update layout
    fig.update_layout(
        title=f'Average Runtime of TV Shows and/or Movies from {start_year} to {end_year}',
        xaxis_title='Year',
        yaxis_title='Average Runtime (minutes)',
        xaxis=dict(tickmode='linear', tickangle=-45, showgrid=True),
        yaxis=dict(showgrid=True),
        template='plotly_white',
        width=1450,  # Set the width of the figure
        height=450  # Set the height of the figure
    )

    # Show the figure
    fig.show()

# Function to create a pie chart for production countries
def pie_country(types, start_year, end_year, genres):
    df_filtered = df[
        (df['type'].isin(types)) &
        (df['release_year'] >= start_year) &
        (df['release_year'] <= end_year) &
        (df['genres'].apply(lambda x: any(item in x for item in genres)))
    ]
    countries_count = df_filtered['production_countries'].explode().value_counts()

    # Check if the number of countries is more than 15
    if len(countries_count) > 15:
        # Get the top 15 countries and the rest as 'Others'
        top_countries = countries_count.head(15)
        others_count = countries_count.iloc[15:].sum()
        # Combine top countries with 'Others'
        countries_count = pd.concat([top_countries, pd.Series({'Others': others_count})])

    # Create the pie chart
    fig = px.pie(countries_count, values=countries_count.values, names=countries_count.index,
                 title=f'Production Countries Distribution from {start_year} to {end_year}')
    
    # Update layout with custom figure size
    fig.update_layout(
        width=1450,  # Set the width of the figure
        height=550  # Set the height of the figure
    )
    
    fig.show()    

# Function to create a scatter plot for IMDb vs TMDb scores with interactive widgets
def scatter_imdb_vs_tmdb(types, start_year, end_year, genres, countries):
    df_filtered = df[
        (df['type'].isin(types)) &
        (df['release_year'] >= start_year) &
        (df['release_year'] <= end_year) &
        (df['genres'].apply(lambda x: any(item in x for item in genres))) &
        (df['production_countries'].apply(lambda x: any(item in x for item in countries)))
    ]

    # Define a custom color scale with darker colors
    color_scale = pc.sequential.Cividis

    fig = px.scatter(df_filtered,
                     x='tmdb_score', 
                     y='imdb_score',
                     size='imdb_votes', 
                     color='release_year', 
                     color_continuous_scale=color_scale, 
                     hover_name='title', 
                     title=f'Imdb vs Tmdb Scores from {start_year} to {end_year}')

    # Update layout with custom figure size
    fig.update_layout(
        width=1450,  # Set the width of the figure
        height=500  # Set the height of the figure
    )
    
    fig.show()

# Function to create a choropleth map for production countries    
def map_num_of_show_andor_movie(types, start_year, end_year, genres):
    # Explode 'production_countries' to plot each country separately
    df_exploded = df.explode('production_countries')

    # Drop rows with NaN values in 'production_countries'
    df_exploded = df_exploded.dropna(subset=['production_countries'])

    # Filter data based on the selected types, year range, and genres
    filtered_df = df_exploded[
        (df_exploded['type'].isin(types)) &
        (df_exploded['release_year'] >= start_year) &
        (df_exploded['release_year'] <= end_year) &
        (df_exploded['genres'].apply(lambda x: any(item in x for item in genres)))
    ]

    # Group by production countries and count the number of titles
    country_counts = filtered_df['production_countries'].value_counts().reset_index()
    country_counts.columns = ['Country', 'Count']

    # Create an interactive map plot
    fig = px.choropleth(
        country_counts,
        locations="Country",
        locationmode="country names",
        color="Count",
        hover_name="Country",
        color_continuous_scale=px.colors.sequential.Plasma,
        title=f"Number of Netflix TV Shows and/or Movies from {start_year} to {end_year}"
    )

    # Update layout with custom figure size
    fig.update_layout(
        width=1450,  # Set the width of the figure
        height=550  # Set the height of the figure
    )
    
    fig.show()
    
# Create unique options with "Select All"
unique_genres = np.append('Select All', df['genres'].explode().dropna().unique())
unique_countries = np.append('Select All', unique_countries)

# Function to handle "Select All" selection
def handle_select_all(widget_value, options):
    if 'Select All' in widget_value:
        return options[1:]  # Return all options except "Select All"
    return widget_value

# Interactive widgets with "Select All" option
type_widget = widgets.SelectMultiple(options=df['type'].unique(), description='Type:', 
                                     layout=widgets.Layout(width='150px', height='50px'))
start_year_widget = widgets.IntSlider(value=2000, min=df['release_year'].min(), max=df['release_year'].max(), step=1, description='Start Year:')
end_year_widget = widgets.IntSlider(value=2020, min=df['release_year'].min(), max=df['release_year'].max(), step=1, description='End Year:')
genre_widget = widgets.SelectMultiple(options=unique_genres, description='Genres:', 
                                     layout=widgets.Layout(width='200px'))
country_widget = widgets.SelectMultiple(options=unique_countries, description='Countries:')

# Control panel with all the widgets
control_panel = HBox([type_widget, VBox([start_year_widget, end_year_widget]), genre_widget, country_widget])

# Buttons for each visualization
button_bar_genre = widgets.Button(description="Bar Genre Distribution", layout=widgets.Layout(width='200px'))
button_line_avg_runtime = widgets.Button(description="Line Avg Runtime", layout=widgets.Layout(width='200px'))
button_pie_country = widgets.Button(description="Pie Country Distribution", layout=widgets.Layout(width='200px'))
button_scatter_imdb_vs_tmdb = widgets.Button(description="Scatter Imdb vs Tmdb", layout=widgets.Layout(width='200px'))
button_map_num_of_show_andor_movie = widgets.Button(description="Map Num of Shows and/or Movies", layout=widgets.Layout(width='250px'))

# Output widgets to display the graphs
output = widgets.Output()

# Function to update the output widget based on the selected visualization
def on_button_clicked(b):
    with output:
        output.clear_output()
        
        # Handle "Select All" for genres and countries
        selected_genres = handle_select_all(genre_widget.value, genre_widget.options)
        selected_countries = handle_select_all(country_widget.value, country_widget.options)
        
        if b == button_bar_genre:
            bar_genre(type_widget.value, start_year_widget.value, end_year_widget.value, selected_countries)
        elif b == button_line_avg_runtime:
            line_avg_runtime(type_widget.value, start_year_widget.value, end_year_widget.value, selected_genres, selected_countries)
        elif b == button_pie_country:
            pie_country(type_widget.value, start_year_widget.value, end_year_widget.value, selected_genres)
        elif b == button_scatter_imdb_vs_tmdb:
            scatter_imdb_vs_tmdb(type_widget.value, start_year_widget.value, end_year_widget.value, selected_genres, selected_countries)
        elif b == button_map_num_of_show_andor_movie:
            map_num_of_show_andor_movie(type_widget.value, start_year_widget.value, end_year_widget.value, selected_genres)

# Attach the on_click event to each button
button_bar_genre.on_click(on_button_clicked)
button_line_avg_runtime.on_click(on_button_clicked)
button_pie_country.on_click(on_button_clicked)
button_scatter_imdb_vs_tmdb.on_click(on_button_clicked)
button_map_num_of_show_andor_movie.on_click(on_button_clicked)

# Display the control panel and the buttons
display(control_panel)
display(HBox([button_bar_genre, button_line_avg_runtime, button_pie_country, button_scatter_imdb_vs_tmdb, button_map_num_of_show_andor_movie]))
display(output)

Output()